<a href="https://colab.research.google.com/github/B-es/Lab_Sii/blob/main/Lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import os
import pandas as pd
import seaborn as sns

# for dirpath, _, filenames in os.walk('/content'):
#   for filename in filenames:
#     pass
#     #print(os.path.join(dirpath, filename))

# with open('/content/test.csv','r') as f1:
#   for i, line in enumerate(f1):
#     print(line.strip())
#     if i == 3:
#       break

train = pd.read_csv('/content/train.csv')
train.head(3)

test = pd.read_csv('/content/test.csv')

#Графики
#sns.countplot(x = 'Pclass', hue = 'Survived', data = train)

#sns.countplot(x = 'Sex', hue = 'Survived', data = train)

#train.isnull().sum()

#Вырезаем ненужные данные
train.drop(columns = 'Cabin', axis = 1, inplace = True)
train['Age'].fillna(train['Age'].mean(), inplace = True)
train.dropna(inplace = True)
# train.isnull().sum()

pd.get_dummies(train['Sex']).head(3)
sex = pd.get_dummies(train['Sex'], drop_first = True)
embarked = pd.get_dummies(train['Embarked'], drop_first = True)
pclass = pd.get_dummies(train['Pclass'], drop_first = True)
train = pd.concat([train, pclass, sex, embarked], axis = 1)
train.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Embarked'], axis = 1, inplace = True)

#Подготовка данных для обучения модели
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_to_scale = ['Age', 'Fare']
scaler.fit(train[cols_to_scale])
train[cols_to_scale] = scaler.transform(train[cols_to_scale])
train.columns = train.columns.map(str)

X_train = train.drop('Survived', axis = 1)
y_train = train['Survived']

#Обучаем модель
from sklearn.linear_model import LogisticRegression

#print(X_train)
model = LogisticRegression()
#print(model)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)

#Матрица ошибок
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_train, y_pred_train)
conf_matrix_df = pd.DataFrame(conf_matrix)

#print(conf_matrix_df)

conf_matrix_labels = pd.DataFrame(conf_matrix, columns = ['Прогноз погиб', 'Прогноз выжил'], index = ['Факт погиб', 'Факт выжил'])

#Должно быть
print(conf_matrix_labels)
print(round((478+237)/(478+237+71+103),3))

#Выдало
from sklearn.metrics import accuracy_score
model_accuracy = accuracy_score(y_train, y_pred_train)
print(round(model_accuracy, 3))

#Построение прогноза на тестовых данных
#test.info()
X_test = test
X_test['Age'].fillna(test['Age'].mean(), inplace = True)
X_test['Fare'].fillna(test['Fare'].mean(), inplace = True)
sex = pd.get_dummies(X_test['Sex'], drop_first = True)
embarked = pd.get_dummies(X_test['Embarked'], drop_first = True)
pclass = pd.get_dummies(X_test['Pclass'], drop_first = True)
X_test = pd.concat([test, pclass, sex, embarked], axis = 1)
X_test.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Cabin', 'Ticket', 'Embarked'], axis = 1, inplace = True)
#X_test.head(3)
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
#X_test.head(3)
X_test.columns = X_test.columns.map(str)
y_pred_test = model.predict(X_test)
print(y_pred_test[:10])

#Result
ids = test['PassengerId']
result = pd.DataFrame({'PassengerId': ids, 'Survived': y_pred_test})
result.head()

result.to_csv('result.csv', index = False)
print("Всё работает")

            Прогноз погиб  Прогноз выжил
Факт погиб            478             71
Факт выжил            103            237
0.804
0.804
[0 0 0 0 1 0 1 0 1 0]
Всё заебись


In [101]:
from google.colab import files

files.download('/content/result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>